# Machine learning and deep learning methods applied to predicting customer status

Purpose: Predict active and not active clients based on the proposed data structure and naive data structure. Here, we implemented Support Vector Machines (SVM), Random Forest (RF), K-nearest Neighbours (KNN) and Lasso.

Author: Gabriel Rodrigues Palma and Rafael de Andrade Moral

# Packages used in the project

In [1]:
# visualisation modules
import matplotlib.pyplot as plt

# Data manipulation modules
import numpy as np
import pandas as pd

# Machine learning modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Deep learning modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import LeakyReLU

# Machine learning packages
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import collections
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Lasso
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import LeaveOneOut

# Additional packages
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import L1
from keras import metrics
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Testing GPU from MacOs
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


# Functions used in the project

In [2]:
def create_dataset(path, response_class):
    ''' This function reads and prepare the datasets for applying the ML and DL methods'''
    data = pd.read_csv(path)
    data = data.drop(columns = ['Unnamed: 0', 'Subject'])
    explanatory_variables = data.drop(columns = response_class)
    response_variables = data[response_class]        
    binarizer = LabelBinarizer()
    response_variables = binarizer.fit_transform(response_variables)
    
    onehot_encoder = OneHotEncoder(sparse=False)
    hot_encode_response_variable = onehot_encoder.fit_transform(np.array(response_variables).reshape(-1, 1))          
    
    return(explanatory_variables, 
           hot_encode_response_variable, response_variables)

def check_zero_division_and_get_rates(cm):
    ''' This functions checks for divisions per zeros in the computation of 
       True and false positive rate based on the confusion matrix array. Also, 
       this function returns the checked rates'''
    
    if any(np.sum(cm, axis = 1)==0):            
            tpr = cm[:,1][1]/(np.sum(cm, axis = 1)[1]+1e-16)
            fpr = cm[:,1][0]/(np.sum(cm, axis = 1)[0]+1e-16)    
    else:           
        rates = cm[:,1]/np.sum(cm, axis = 1)    
        fpr = rates[0]    
        tpr = rates[1]
    if np.isnan(tpr):        
        tpr=0
        
    return(tpr, fpr)

def check_and_compute_rates(predictions, 
                            classes, 
                            cm):    
    ''' This function returns the values of true and false positive rate for special cases
       where the division is not possible to obtain automatically based on the confusion
       matrix array provided by sklearn'''
        
    if (sum(predictions) == 0 and sum(classes) == 0):        
        fpr = 0
        tpr = 0
    elif (sum(predictions) == len(predictions) and sum(classes) == len(classes)):        
        fpr = 0
        tpr = 1
    else:            
        tpr, fpr = check_zero_division_and_get_rates(cm)
        
    return(tpr, fpr)
    
def get_rates(y_pred,
              y_true):
    ''' This function get the true and false positive rates based on the 
       predictied'''
            
    cm = confusion_matrix(y_true = y_true, y_pred = y_pred)
    tpr, fpr = check_and_compute_rates(y_pred, y_true, cm)
        
    return(tpr, fpr)

def get_rates_by_cross_validation(raw_data):
    ''' This function obtains the accuracy, true and false positive rates 
       based on the cross k-fold cross validation'''
            
    for train_index, test_index in KFold(n_splits=5, shuffle=True).split(patterns):
        
        x_train, x_test = patterns[train_index], patterns[test_index]
        y_train, y_test = classes[train_index], classes[test_index]
                
        pbp_predictions = pbp_prediction(patterns_array=x_test, 
                                         clustered_patterns = clustered_patterns, 
                                         d_base = d_base, alpha = alpha, 
                   outbreak_p_means = prediction.obtain_p_means_with_distance,
                   outbreak_prediction = prediction.predict_with_distance)
        rates = get_rates(predictions = pbp_predictions, classes = y_test)
        tpr.append(rates[0])
        fpr.append(rates[1])
    
    return(np.mean(tpr[tpr!=np.nan]), np.mean(fpr[fpr!=np.nan]))

def get_statistics(y_pred, y_true):
    ''' This function obtains the following statistics: Accuracy, True positive rate and 
       False positive rate'''

    auc = roc_auc_score(y_score = y_pred,
              y_true = y_true)
    acc = accuracy_score(y_pred = y_pred,
                         y_true = y_true)
    rates = get_rates(y_pred = y_pred,
              y_true = y_true)
    tpr = rates[0]
    fpr = rates[1]
    return([acc, tpr, fpr, auc])

# def get_methods_performance(X_train, X_test, 
#                             y_train, y_test, 
#                             raw_data):
#     ''' This function obtain the performance of each selected model'''
#     kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
#     # Support Vector Machine -----
#     ## Non-linear
#     nonlinear_svm = svm.NuSVC(gamma="auto")
#     # Create the parameter space
#     params = {"kernel": ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
#               "gamma": ['auto', 'scale'],
#              "degree": np.arange(1, 10, 1)}
#     nonlinear_svm_cv = RandomizedSearchCV(nonlinear_svm, params, cv=kf)
#     nonlinear_svm_cv.fit(X_train, y_train)
#     nonlinear_svm_predictions = nonlinear_svm_cv.predict(X_test)
    
#     ## Polinomial
#     polinomial_svm = svm.SVC(kernel='poly', degree = 3)
#     polinomial_svm.fit(X_train, y_train)
#     polinomial_svm_predictions = polinomial_svm.predict(X_test)
    
#     # Random forest -----
#     rf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators = 1000)
#     rf.fit(X_train, y_train)
#     random_forest_predictions = rf.predict(X_test)
    
#     # KNN -----
#     knn = KNeighborsClassifier(n_neighbors=1)
#     knn.fit(X_train, y_train)
#     knn_predictions = knn.predict(X_test)
    
#     # Lasso -----
#     lasso = Lasso(alpha=0.02)
#     lasso.fit(X_train, y_train)
#     lasso_predictions = np.round(lasso.predict(X_test))
    
#     # Deep Neural Network -----
# #     if raw_data==True:
# #         dnn = keras.models.load_model('DNN/DNN_model_params.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))
# #     elif raw_data == 'combined':
# #         dnn = keras.models.load_model('DNN/DNN_model_fulldata.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))
# #     else:    
# #         dnn = keras.models.load_model('DNN/DNN_model_hmm.h5')
# #         dnn_predictions = np.round(dnn.predict(X_test))

    
#     # Obtaining statistics -----
#     polynomial_svm_performance = get_statistics(y_pred = polinomial_svm_predictions,
#                                         y_true = y_test)
#     nonlinear_svm_performance = get_statistics(y_pred = nonlinear_svm_predictions,
#                                        y_true = y_test)
#     random_forest_performance = get_statistics(y_pred = random_forest_predictions,
#                                        y_true = y_test)
#     knn_performance = get_statistics(y_pred = knn_predictions,
#                                        y_true = y_test)    
#     lasso_performance = get_statistics(y_pred = lasso_predictions,
#                                        y_true = y_test)
# #     dnn_performance = get_statistics(y_pred = dnn_predictions,
# #                                        y_true = y_test)

#     return(polynomial_svm_performance, nonlinear_svm_performance, 
#            random_forest_performance, knn_performance, 
#            lasso_performance)
    
# def get_results_data(explanatory_variables,                             
#                      response_variables,                                                         
#                      test_size, raw_data):
#     ''' This function obtain the performance of the ML and DL methods based on
#        the prediction of client status'''
#     X_train, X_test, y_train, y_test = train_test_split(explanatory_variables, 
#                                                         response_variables, 
#                                                         test_size = test_size, 
#                                                         random_state = 42)
   
#     # Obtaining methods performance
#     polynomial_svm_performance, nonlinear_svm_performance, \
#     random_forest_performance, knn_performance, \
#     lasso_performance = get_methods_performance(X_train, X_test, 
#                                               y_train, y_test, 
#                                               raw_data)
        
    
#     model_outputs_performance = pd.DataFrame({'Statistics':['Accuracy', 
#                                                             'True Positive Rate', 
#                                                             'False Positive Rate', 
#                                                             'AUROC'],
#                                               'Polinomial SVM': [polynomial_svm_performance[0], 
#                                                                  polynomial_svm_performance[1], 
#                                                                  polynomial_svm_performance[2],
#                                                                 polynomial_svm_performance[3]], 
#                                               'Non linear SVM': [nonlinear_svm_performance[0], 
#                                                                  nonlinear_svm_performance[1], 
#                                                                  nonlinear_svm_performance[2], 
#                                                                 nonlinear_svm_performance[3]], 
#                                               'Random Forest': [random_forest_performance[0], 
#                                                                 random_forest_performance[1], 
#                                                                 random_forest_performance[2], 
#                                                                random_forest_performance[3]], 
#                                               'KNN': [knn_performance[0], 
#                                                       knn_performance[1], 
#                                                       knn_performance[2], 
#                                                       knn_performance[3]],                                                
#                                               'Lasso': [lasso_performance[0], 
#                                                         lasso_performance[1], 
#                                                         lasso_performance[2], 
#                                                         lasso_performance[3]]})

#     return(model_outputs_performance)
    

In [3]:
def get_methods_predictions(X_train, X_test, 
                            y_train, y_test, 
                            data_type):
    ''' This function obtain the performance of each selected model'''
    kf = KFold(n_splits = 5, random_state = 42, shuffle = True)
    # Support Vector Machine -----
    ## Non-linear
    nonlinear_svm = svm.NuSVC()
    
    nonlinear_svm.fit(X_train, y_train)
    
    nonlinear_svm_predictions = nonlinear_svm.predict(X_test)
    
    ## Polinomial
    polinomial_svm = svm.SVC(kernel='poly', degree = 3)
    polinomial_svm.fit(X_train, y_train)
    polinomial_svm_predictions = polinomial_svm.predict(X_test)
    
    # Random forest -----
    rf = RandomForestClassifier(max_depth=5, random_state=0, n_estimators = 1000)
    rf.fit(X_train, y_train)
    random_forest_predictions = rf.predict(X_test)
    
    # KNN -----
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train, y_train)
    knn_predictions = knn.predict(X_test)
    
    # Lasso -----
    lasso = Lasso(alpha=0.02)
    lasso.fit(X_train, y_train)
    lasso_predictions = np.round(lasso.predict(X_test))
    
    # Deep Neural Network -----
    if data_type == 'All':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(50,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
        
    elif data_type == 'm2':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(13,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))

    elif data_type == 'm3':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(16,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
        
    elif data_type == 'm4':
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(19,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
        
        
    else:    
        dnn = Sequential()
        dnn.add(Dense(100, input_shape=(22,)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(200, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(150, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(46, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(20, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(10, activation = LeakyReLU(alpha=0.4)))
        dnn.add(Dense(1, activation = 'sigmoid'))
        dnn.compile(optimizer = 'adam', 
                      loss = 'mse', 
                     metrics = ['accuracy', 'TruePositives', 'FalsePositives', 'FalseNegatives', 'TrueNegatives'])
        dnn.fit(X_train, y_train, batch_size = 10, epochs=100, verbose = False)
        dnn_predictions = np.round(dnn.predict(X_test))
    

    return(polinomial_svm_predictions, nonlinear_svm_predictions, 
           random_forest_predictions, knn_predictions, 
           lasso_predictions, dnn_predictions)
def get_methods_predictions_loo(polinomial_svm_predictions, nonlinear_svm_predictions, 
                                random_forest_predictions, knn_predictions, 
                                lasso_predictions, dnn_predictions, 
                                reponse_variable):
    
    '''This function obtain the methods performance based on their predictions for the leave one out cross valition'''
    polynomial_svm_performance = get_statistics(y_pred = polinomial_svm_predictions,
                                    y_true = reponse_variable)
    nonlinear_svm_performance = get_statistics(y_pred = nonlinear_svm_predictions,
                                       y_true = reponse_variable)
    random_forest_performance = get_statistics(y_pred = random_forest_predictions,
                                       y_true = reponse_variable)
    knn_performance = get_statistics(y_pred = knn_predictions,
                                       y_true = reponse_variable)    
    lasso_performance = get_statistics(y_pred = lasso_predictions,
                                       y_true = reponse_variable)
    dnn_performance = get_statistics(y_pred = dnn_predictions,
                                        y_true = reponse_variable)
    return(polynomial_svm_performance, nonlinear_svm_performance, 
           random_forest_performance, knn_performance, 
           lasso_performance, dnn_performance)

    
def get_results_data_loo(explanatory_variables,                         
                         response_variables,                                                         
                         data_type):
    ''' This function obtain the performance of the ML and DL methods based on
       the prediction of client status based on leave one out cross validation'''

    loo = LeaveOneOut()
    loo.get_n_splits(explanatory_variables)
    polinomial_svm_predictions, nonlinear_svm_predictions, \
           random_forest_predictions, knn_predictions, \
           lasso_predictions, dnn_predictions = ([], [], [], [], [], [])
    
    for train_index, test_index in loo.split(explanatory_variables):
        X_train, X_test = explanatory_variables[train_index], explanatory_variables[test_index]
        y_train, y_test = response_variables[train_index], response_variables[test_index]                
             
        ## Correcting errors related to column vector (n_observations, ) to (, n_observations)
        y_train = y_train.ravel() 
        y_test = y_test.ravel()        
    
        polynomial_svm_prediction, nonlinear_svm_prediction, \
        random_forest_prediction, knn_prediction, \
        lasso_prediction, dnn_prediction = get_methods_predictions(X_train, X_test, 
                                                   y_train, y_test, 
                                                   data_type)
        
        polinomial_svm_predictions.append(polynomial_svm_prediction[0])
        nonlinear_svm_predictions.append(nonlinear_svm_prediction[0])
        random_forest_predictions.append(random_forest_prediction[0])
        knn_predictions.append(knn_prediction[0])        
        lasso_predictions.append(lasso_prediction[0]) 
        dnn_predictions.append(dnn_prediction[0])
 
    polynomial_svm_performance, nonlinear_svm_performance, \
       random_forest_performance, knn_performance, \
       lasso_performance, dnn_performance = get_methods_predictions_loo(polinomial_svm_predictions, nonlinear_svm_predictions, 
                            random_forest_predictions, knn_predictions, 
                            lasso_predictions, dnn_predictions, 
                            list(response_variables.flatten()))
    
    model_outputs_performance = pd.DataFrame({'Statistics':['Accuracy', 
                                                            'True Positive Rate', 
                                                            'False Positive Rate', 
                                                            'AUROC'],
                                              'Polinomial SVM': [polynomial_svm_performance[0], 
                                                                 polynomial_svm_performance[1], 
                                                                 polynomial_svm_performance[2],
                                                                polynomial_svm_performance[3]], 
                                              'Non linear SVM': [nonlinear_svm_performance[0], 
                                                                 nonlinear_svm_performance[1], 
                                                                 nonlinear_svm_performance[2], 
                                                                nonlinear_svm_performance[3]], 
                                              'Random Forest': [random_forest_performance[0], 
                                                                random_forest_performance[1], 
                                                                random_forest_performance[2], 
                                                               random_forest_performance[3]], 
                                              'KNN': [knn_performance[0], 
                                                      knn_performance[1], 
                                                      knn_performance[2], 
                                                      knn_performance[3]],                                                
                                              'Lasso': [lasso_performance[0], 
                                                        lasso_performance[1], 
                                                        lasso_performance[2], 
                                                        lasso_performance[3]], 
                                             'DNN': [dnn_performance[0], 
                                                        dnn_performance[1], 
                                                        dnn_performance[2], 
                                                        dnn_performance[3]]})

    return(model_outputs_performance)

# Importing Datasets

Before obtaining the performance of the learning algorithms selected for this paper, we need to import the datasets related to the Hidden Markov Models features and the peak features.

## Minmax scale Midline EEG data Trial 1

In [4]:
all_Minmax_scale_explanatory_variables_t1, \
  all_Minmax_scale_hot_encode_response_variable_t1, all_Minmax_scale_response_variable_t1 = create_dataset(path = '../../output_data/New_data/CoordinatesPlusEEGData/MinmaxscalePlusCoordinateTrial1.csv', 
                                                                                           response_class = 'Class')

In [5]:
all_Minmax_scale_explanatory_variables_t1.head()

,MM_Intercept,MM_Hess,average_peak_magnitude,average_peak_curvature,hmm2_mean.1,hmm2_mean.2,hmm2_sd.1,hmm2_sd.2,hmm2_p1,hmm2_p2,...,hmm5_sd.5,hmm5_p1,hmm5_p2,hmm5_p3,hmm5_p4,hmm5_p5,dis_sum,angle_sum,average_speed,idle_time
0,0.052283,-10.740852,0.329249,-0.000296,0.348,0.216,0.065,0.037,0.515900,0.484100,...,0.012,0.180843,0.240996,0.243103,0.144828,0.190230,0.529958,0.146742,0.106868,0.347826
1,0.052178,-10.722025,0.340976,-0.000324,0.209,0.364,0.048,0.092,0.530876,0.469124,...,0.015,0.156503,0.214782,0.282709,0.138557,0.207449,0.592253,0.355012,0.262695,0.065217
2,-0.030231,6.213944,0.230614,-0.000272,0.269,0.147,0.064,0.032,0.404959,0.595041,...,0.009,0.229291,0.181818,0.179896,0.210456,0.198539,0.564102,0.210705,0.177640,0.173913
3,-0.000309,0.064725,0.239325,-0.000217,0.264,0.155,0.045,0.034,0.440678,0.559322,...,0.022,0.190485,0.208398,0.214176,0.224576,0.162365,0.565466,0.495757,0.259048,0.152174
4,0.094943,-19.507132,0.403699,-0.000361,0.275,0.459,0.053,0.082,0.627565,0.372435,...,0.014,0.237708,0.252226,0.111692,0.255517,0.142857,0.524729,0.535833,0.212309,0.282609


## Minmax scale Midline EEG data Trial 12

In [6]:
all_Minmax_scale_explanatory_variables_t12, \
  all_Minmax_scale_hot_encode_response_variable_t12, all_Minmax_scale_response_variable_t12 = create_dataset(path = '../../output_data/New_data/CoordinatesPlusEEGData/MinmaxscalePlusCoordinateTrial12.csv', 
                                                                                           response_class = 'Class')

In [7]:
all_Minmax_scale_explanatory_variables_t12.head()

,MM_Intercept,MM_Hess,average_peak_magnitude,average_peak_curvature,hmm2_mean.1,hmm2_mean.2,hmm2_sd.1,hmm2_sd.2,hmm2_p1,hmm2_p2,...,hmm5_sd.5,hmm5_p1,hmm5_p2,hmm5_p3,hmm5_p4,hmm5_p5,dis_sum,angle_sum,average_speed,idle_time
0,0.061130,0.705040,0.372955,-0.000386,0.394,0.253,0.073,0.048,0.315457,0.684543,...,0.067,0.194532,0.219769,0.256572,0.170347,0.158780,0.207386,0.165184,0.502991,0.029412
1,0.020163,-88.391867,0.351785,-0.000365,0.224,0.442,0.029,0.149,0.590447,0.409553,...,0.153,0.149390,0.257114,0.183943,0.295732,0.113821,0.335386,0.178138,0.692404,0.000000
2,-0.044913,-101.184206,0.212457,-0.000230,0.126,0.261,0.022,0.090,0.470644,0.529356,...,0.091,0.129735,0.188447,0.133523,0.357008,0.191288,0.106150,0.045917,0.161536,0.264706
3,-0.001816,-48.016730,0.266135,-0.000272,0.308,0.171,0.059,0.031,0.423729,0.576271,...,0.048,0.095450,0.329170,0.116860,0.266726,0.191793,0.211527,0.905673,0.523002,0.088235
4,0.037201,-95.792486,0.377899,-0.000374,0.418,0.236,0.080,0.059,0.412071,0.587929,...,0.068,0.262227,0.234131,0.123829,0.138398,0.241415,0.342820,0.012521,0.702598,0.000000


# Obtaining learning algorithms performance based on Leave one out cross validation

Now, we are obtaining accuracy, trus and false positive rates for Polinomial and no linear Support Vector Machines, Random Forests, K-neirest neighbors and Lasso learning algorithms.

## Minmax scale EEG data Trial 1

For the raw Midline EEG data, we are going to analise train and analyse the variables in the following way:

    1. All the features;
    2. The features for HMM with 2 parameters;
    3. The features for HMM with 3 parameters;
    4. The features for HMM with 4 parameters;
    5. The features for HMM with 5 parameters;

### All the features

In [8]:
get_results_data_loo(explanatory_variables = np.array(all_Minmax_scale_explanatory_variables_t1),                                                  
                     response_variables = all_Minmax_scale_response_variable_t1,                                                                           
                     data_type = 'All')

Metal device set to: Apple M1


2023-04-06 15:51:33.042307: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-06 15:51:33.042645: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-06 15:51:33.361666: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-06 15:51:33.970584: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 92ms/step


2023-04-06 15:51:38.912684: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 15:51:39.892129: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 62ms/step


...

1/1 [==============================] - 0s 105ms/step


2023-04-06 15:55:38.750837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 15:55:39.768210: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 88ms/step


2023-04-06 15:55:45.261823: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.564103,0.743590,0.641026,0.589744,0.692308,0.692308
1,True Positive Rate,1.000000,0.636364,0.727273,0.545455,0.818182,0.727273
2,False Positive Rate,1.000000,0.117647,0.470588,0.352941,0.470588,0.352941
3,AUROC,0.500000,0.759358,0.628342,0.596257,0.673797,0.687166


### The features for HMM with 2 parameters

In [9]:
selected_columns_m2 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm2_mean.1', 
                    'hmm2_mean.2', 'hmm2_sd.1', 'hmm2_sd.2', 
                    'hmm2_p1', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m2_array = [elem in selected_columns_m2 for elem in all_Minmax_scale_explanatory_variables_t1.columns]

In [10]:
all_Minmax_scale_explanatory_variables_t1_m2 = all_Minmax_scale_explanatory_variables_t1[selected_columns_m2]

In [11]:
get_results_data_loo(explanatory_variables = np.array(all_Minmax_scale_explanatory_variables_t1_m2),                                                  
                     response_variables = all_Minmax_scale_response_variable_t1,                                                                           
                     data_type = 'm2')

2023-04-06 15:55:46.550024: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 122ms/step


2023-04-06 15:55:52.203814: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 15:55:53.196975: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 93ms/step


2023-04-06 15:55:58.627497: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 15:56:00.028080: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 213ms/step


2023-04-06 16:00:32.697103: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:00:33.983883: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 194ms/step


2023-04-06 16:00:41.892695: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.564103,0.743590,0.717949,0.589744,0.384615,0.717949
1,True Positive Rate,1.000000,0.636364,0.727273,0.545455,0.636364,0.727273
2,False Positive Rate,1.000000,0.117647,0.294118,0.352941,0.941176,0.294118
3,AUROC,0.500000,0.759358,0.716578,0.596257,0.347594,0.716578


### The features for HMM with 3 parameters

In [12]:
selected_columns_m3 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm3_mean.1', 'hmm3_mean.2', 
                    'hmm3_mean.3', 'hmm3_sd.1', 
                    'hmm3_sd.2', 'hmm3_sd.3', 'hmm3_p1',
                    'hmm3_p2', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m3_array = [elem in selected_columns_m3 for elem in all_Minmax_scale_explanatory_variables_t1.columns]

In [13]:
all_Minmax_scale_explanatory_variables_t1_m3 = all_Minmax_scale_explanatory_variables_t1[selected_columns_m3]

In [14]:
get_results_data_loo(explanatory_variables = np.array(all_Minmax_scale_explanatory_variables_t1_m3),                                                  
                     response_variables = all_Minmax_scale_response_variable_t1,                                                                           
                     data_type = 'm3')

2023-04-06 16:00:43.417291: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:00:50.825413: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 468ms/step


2023-04-06 16:00:52.258035: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 157ms/step


2023-04-06 16:00:59.734938: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:01:01.158239: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 201ms/step


2023-04-06 16:06:04.085510: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:06:05.203146: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 197ms/step


2023-04-06 16:06:12.829256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.564103,0.743590,0.717949,0.589744,0.384615,0.743590
1,True Positive Rate,1.000000,0.636364,0.727273,0.545455,0.636364,0.727273
2,False Positive Rate,1.000000,0.117647,0.294118,0.352941,0.941176,0.235294
3,AUROC,0.500000,0.759358,0.716578,0.596257,0.347594,0.745989


### The features for HMM with 4 parameters

In [8]:
selected_columns_m4 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm4_mean.1', 'hmm4_mean.2', 'hmm4_mean.3',
                       'hmm4_mean.4', 'hmm4_sd.1', 'hmm4_sd.2', 'hmm4_sd.3', 'hmm4_sd.4',
                       'hmm4_p1', 'hmm4_p2', 'hmm4_p3', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m4_array = [elem in selected_columns_m4 for elem in all_Minmax_scale_explanatory_variables_t1.columns]

In [9]:
all_Minmax_scale_explanatory_variables_t1_m4 = all_Minmax_scale_explanatory_variables_t1[selected_columns_m4]

In [10]:
get_results_data_loo(explanatory_variables = np.array(all_Minmax_scale_explanatory_variables_t1_m4),                                               
                     response_variables = all_Minmax_scale_response_variable_t1,                                                                           
                     data_type = 'm4')

Metal device set to: Apple M1


2023-04-06 16:10:59.641052: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-06 16:10:59.641159: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-06 16:10:59.831732: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-06 16:11:00.383166: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 94ms/step


2023-04-06 16:11:05.276272: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:11:06.234575: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 62ms/step


...

1/1 [==============================] - 0s 88ms/step


2023-04-06 16:14:58.749580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:15:00.099917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 101ms/step


2023-04-06 16:15:05.637905: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.564103,0.743590,0.743590,0.589744,0.692308,0.820513
1,True Positive Rate,1.000000,0.636364,0.818182,0.545455,0.818182,0.818182
2,False Positive Rate,1.000000,0.117647,0.352941,0.352941,0.470588,0.176471
3,AUROC,0.500000,0.759358,0.732620,0.596257,0.673797,0.820856


### The features for HMM with 5 parameters

In [11]:
selected_columns_m5 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm5_mean.1',
                       'hmm5_mean.2', 'hmm5_mean.3', 'hmm5_mean.4', 'hmm5_mean.5', 'hmm5_sd.1',
                       'hmm5_sd.2', 'hmm5_sd.3', 'hmm5_sd.4', 'hmm5_sd.5', 'hmm5_p1',
                       'hmm5_p2', 'hmm5_p3', 'hmm5_p4', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m5_array = [elem in selected_columns_m5 for elem in all_Minmax_scale_explanatory_variables_t1.columns]

In [12]:
all_Minmax_scale_explanatory_variables_t1_m5 = all_Minmax_scale_explanatory_variables_t1[selected_columns_m5]

In [13]:
get_results_data_loo(explanatory_variables = np.array(all_Minmax_scale_explanatory_variables_t1_m5),                                                  
                     response_variables = all_Minmax_scale_response_variable_t1,                                                                           
                     data_type = 'm5')

2023-04-06 16:15:06.713561: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 93ms/step


2023-04-06 16:15:12.126244: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:15:13.484421: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 88ms/step


2023-04-06 16:15:18.890360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:15:19.886512: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 137ms/step


2023-04-06 16:19:53.440608: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:19:54.543997: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 138ms/step


2023-04-06 16:20:00.908936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.564103,0.743590,0.743590,0.589744,0.512821,0.769231
1,True Positive Rate,1.000000,0.636364,0.818182,0.545455,0.727273,0.727273
2,False Positive Rate,1.000000,0.117647,0.352941,0.352941,0.764706,0.176471
3,AUROC,0.500000,0.759358,0.732620,0.596257,0.481283,0.775401


## Minmax scale EEG data Trial 12

For the Minmax scale EEG data, we are going to analise train and analyse the variables in the following way:

    1. All the features;
    2. The features for HMM with 2 parameters;
    3. The features for HMM with 3 parameters;
    4. The features for HMM with 4 parameters;
    5. The features for HMM with 5 parameters;

### All the features

In [14]:
get_results_data_loo(explanatory_variables = np.array(all_Minmax_scale_explanatory_variables_t12),                                                  
                     response_variables = all_Minmax_scale_response_variable_t12,                                                                           
                     data_type = 'All')

2023-04-06 16:20:02.408534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 145ms/step


2023-04-06 16:20:09.022402: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:20:10.152475: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 126ms/step


2023-04-06 16:20:16.403728: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:20:17.791557: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 176ms/step


2023-04-06 16:25:10.832989: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:25:12.250865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 181ms/step


2023-04-06 16:25:19.790887: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.589744,0.538462,0.589744,0.512821,0.666667,0.564103
1,True Positive Rate,1.000000,0.681818,0.727273,0.590909,0.909091,0.727273
2,False Positive Rate,0.941176,0.647059,0.588235,0.588235,0.647059,0.647059
3,AUROC,0.529412,0.517380,0.569519,0.501337,0.631016,0.540107


### The features for HMM with 2 parameters

In [15]:
selected_columns_m2 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm2_mean.1', 
                       'hmm2_mean.2', 'hmm2_sd.1', 'hmm2_sd.2', 
                       'hmm2_p1', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m2_array = [elem in selected_columns_m2 for elem in all_Minmax_scale_explanatory_variables_t12.columns]

In [16]:
all_Minmax_scale_explanatory_variables_t12_m2 = all_Minmax_scale_explanatory_variables_t12[selected_columns_m2]

In [17]:
get_results_data_loo(explanatory_variables = np.array(all_Minmax_scale_explanatory_variables_t12_m2),                                                 
                     response_variables = all_Minmax_scale_response_variable_t12,                                                                           
                     data_type = 'm2')

2023-04-06 16:25:20.938153: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 182ms/step


2023-04-06 16:25:28.503435: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:25:29.905862: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 194ms/step


2023-04-06 16:25:37.627600: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:25:38.687641: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

1/1 [==============================] - 0s 232ms/step


2023-04-06 16:31:03.572709: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 203ms/step


2023-04-06 16:31:11.536960: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.589744,0.538462,0.589744,0.512821,0.666667,0.589744
1,True Positive Rate,1.000000,0.681818,0.636364,0.590909,0.909091,0.590909
2,False Positive Rate,0.941176,0.647059,0.470588,0.588235,0.647059,0.411765
3,AUROC,0.529412,0.517380,0.582888,0.501337,0.631016,0.589572


### The features for HMM with 3 parameters

In [18]:
selected_columns_m3 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                    'average_peak_curvature', 'hmm3_mean.1', 'hmm3_mean.2', 
                    'hmm3_mean.3', 'hmm3_sd.1', 
                    'hmm3_sd.2', 'hmm3_sd.3', 'hmm3_p1',
                    'hmm3_p2', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m3_array = [elem in selected_columns_m3 for elem in all_Minmax_scale_explanatory_variables_t12.columns]

In [19]:
all_Minmax_scale_explanatory_variables_t12_m3 = all_Minmax_scale_explanatory_variables_t12[selected_columns_m3]

In [20]:
get_results_data_loo(explanatory_variables = np.array(all_Minmax_scale_explanatory_variables_t12_m3),                                                
                     response_variables = all_Minmax_scale_response_variable_t12,                                                                           
                     data_type = 'm3')

2023-04-06 16:31:13.143688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:31:21.432493: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 240ms/step


2023-04-06 16:31:22.545520: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 194ms/step


2023-04-06 16:31:30.515939: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:31:31.969511: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-06 16:37:39.490019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:37:50.391434: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 402ms/step


2023-04-06 16:37:52.052385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:38:03.589679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 385ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.589744,0.538462,0.666667,0.512821,0.666667,0.692308
1,True Positive Rate,1.000000,0.681818,0.681818,0.590909,0.909091,0.727273
2,False Positive Rate,0.941176,0.647059,0.352941,0.588235,0.647059,0.352941
3,AUROC,0.529412,0.517380,0.664439,0.501337,0.631016,0.687166


### The features for HMM with 4 parameters

In [21]:
selected_columns_m4 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm4_mean.1', 'hmm4_mean.2', 'hmm4_mean.3',
                       'hmm4_mean.4', 'hmm4_sd.1', 'hmm4_sd.2', 'hmm4_sd.3', 'hmm4_sd.4',
                       'hmm4_p1', 'hmm4_p2', 'hmm4_p3', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m4_array = [elem in selected_columns_m4 for elem in all_Minmax_scale_explanatory_variables_t12.columns]

In [22]:
all_Minmax_scale_explanatory_variables_t12_m4 = all_Minmax_scale_explanatory_variables_t12[selected_columns_m4]

In [23]:
get_results_data_loo(explanatory_variables = np.array(all_Minmax_scale_explanatory_variables_t12_m4),                                                  
                     response_variables = all_Minmax_scale_response_variable_t12,                                                                           
                     data_type = 'm4')

2023-04-06 16:38:04.985858: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:38:16.751647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 388ms/step


2023-04-06 16:38:18.463963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:38:30.386661: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 378ms/step


2023-04-06 16:38:31.664851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:38:43.466019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-06 16:47:12.372127: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:47:25.253516: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 426ms/step


2023-04-06 16:47:26.571039: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:47:40.334466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 482ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.589744,0.589744,0.692308,0.512821,0.666667,0.564103
1,True Positive Rate,1.000000,0.772727,0.772727,0.590909,0.909091,0.590909
2,False Positive Rate,0.941176,0.647059,0.411765,0.588235,0.647059,0.470588
3,AUROC,0.529412,0.562834,0.680481,0.501337,0.631016,0.560160


### The features for HMM with 5 parameters

In [24]:
selected_columns_m5 = ['MM_Intercept', 'MM_Hess', 'average_peak_magnitude', 
                       'average_peak_curvature', 'hmm5_mean.1',
                       'hmm5_mean.2', 'hmm5_mean.3', 'hmm5_mean.4', 'hmm5_mean.5', 'hmm5_sd.1',
                       'hmm5_sd.2', 'hmm5_sd.3', 'hmm5_sd.4', 'hmm5_sd.5', 'hmm5_p1',
                       'hmm5_p2', 'hmm5_p3', 'hmm5_p4', 'dis_sum', 'angle_sum', 'average_speed', 'idle_time']
selected_columns_m5_array = [elem in selected_columns_m5 for elem in all_Minmax_scale_explanatory_variables_t12.columns]

In [25]:
all_Minmax_scale_explanatory_variables_t12_m5 = all_Minmax_scale_explanatory_variables_t12[selected_columns_m5]

In [26]:
get_results_data_loo(explanatory_variables = np.array(all_Minmax_scale_explanatory_variables_t12_m5),                                                  
                     response_variables = all_Minmax_scale_response_variable_t12,                                                                           
                     data_type = 'm5')

2023-04-06 16:47:42.396166: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:47:56.257256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 507ms/step


2023-04-06 16:47:57.653110: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:48:11.671479: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 625ms/step


2023-04-06 16:48:13.725678: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:48:27.801759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-06 16:57:11.224501: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:57:25.269748: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 507ms/step


2023-04-06 16:57:27.057084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-06 16:57:41.118611: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 478ms/step


,Statistics,Polinomial SVM,Non linear SVM,Random Forest,KNN,Lasso,DNN
0,Accuracy,0.589744,0.564103,0.717949,0.512821,0.666667,0.512821
1,True Positive Rate,1.000000,0.727273,0.818182,0.590909,0.909091,0.590909
2,False Positive Rate,0.941176,0.647059,0.411765,0.588235,0.647059,0.588235
3,AUROC,0.529412,0.540107,0.703209,0.501337,0.631016,0.501337
